<a href="https://colab.research.google.com/github/HaiderAli998/StructuredAssessments/blob/main/StructuredTask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y openai
!pip uninstall -y openai  # run twice in case multiple dists
!pip install --no-cache-dir "openai==1.96.1" "httpx==0.28.1" "jsonschema==4.22.0" "tiktoken==0.7.0" "python-dotenv==1.0.1"


Found existing installation: openai 1.96.1
Uninstalling openai-1.96.1:
  Successfully uninstalled openai-1.96.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.5/757.5 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 183.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 102.9 MB/s eta 0:00:00
  Attempting uninstall: tiktoken
    Found existing installation: tiktoken 0.9.0
    Uninstalling tiktoken-0.9.0:
      Successfully uninstalled tiktoken-0.9.0
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.24.0
    Uninstalling jsonschema-4.24.0:
      Successfully uninstalled jsonschema-4.24.0


In [2]:
import getpass, os

api_key = getpass.getpass("Enter OPENAI_API_KEY (input hidden): ")
os.environ["OPENAI_API_KEY"] = api_key.strip()

# Basic sanity
assert os.environ["OPENAI_API_KEY"].startswith("sk-"), "Key format looks off."

MODEL = "gpt-4o-mini"  # override if desired

Enter OPENAI_API_KEY (input hidden): ··········


In [3]:
PROMPT_TEMPLATE = """You are an information extraction engine.
Input: A raw textual inpatient clinical/fax document (possibly noisy OCR, repeated sections, headers/footers, pagination artifacts).
Task: Extract a single patient record and output ONLY strict JSON (UTF-8, no extra text) matching the provided JSON Schema.

Rules:
- If a field is truly absent, use null (except current_medications which is "" if none).
- Normalize dates to ISO 8601 YYYY-MM-DD.
- Trim all string values.
- current_medications: semicolon-separated list of ACTIVE / continued (“c/w”, “continue”, “currently on”, “home meds to continue”) meds with dose & frequency if present; ignore historical, discontinued, trial, allergy, PRN (unless explicitly currently ordered) entries. If none, "".
- Names may appear “Last, First”. Split into first_name and last_name correctly.
- DOB may have 2-digit year; pivot 00–29 => 2000–2029, 30–99 => 1900–1999.
- Admission date: explicit “Admit”, “Admission Date”. Pick earliest applicable for current encounter.
- Discharge date: only if explicit; blank → null.
- insurance_id: patient insurance / subscriber number (not group or plan).
- patient_id: if a valid UUID appears in doc, use first one; else generate a RFC4122 v4 UUID (do NOT reuse across different patients).
- Output must validate the schema. No extra keys. All non-null fields are strings.
- Re-validate internally: date regex ^\\d{{4}}-\\d{{2}}-\\d{{2}}$ or null; UUID pattern; no extra properties. If invalid, fix and re-emit.

JSON Schema:
{{ "$schema":"http://json-schema.org/draft-07/schema#","type":"object","required":["patient_id","first_name","last_name","date_of_birth","insurance_id","admission_date","discharge_date","current_medications"],"properties":{"patient_id":{"type":"string","pattern":"^[0-9a-fA-F]{{8}}-[0-9a-fA-F]{{4}}-[1-5][0-9a-fA-F]{{3}}-[89abAB][0-9a-fA-F]{{3}}-[0-9a-fA-F]{{12}}$"},"first_name":{"type":["string","null"]},"last_name":{"type":["string","null"]},"date_of_birth":{"type":["string","null"],"pattern":"^\\d{{4}}-\\d{{2}}-\\d{{2}}$"},"insurance_id":{"type":["string","null"]},"admission_date":{"type":["string","null"],"pattern":"^\\d{{4}}-\\d{{2}}-\\d{{2}}$"},"discharge_date":{"type":["string","null"],"pattern":"^\\d{{4}}-\\d{{2}}-\\d{{2}}$"},"current_medications":{"type":"string"}},"additionalProperties":false }}

Return ONLY the JSON object.

DOCUMENT_START
{DOCUMENT_TEXT}
DOCUMENT_END
"""

SCHEMA = {
  "$schema": "http://json-schema.org/draft-07/schema#",
  "type": "object",
  "required": [
    "patient_id","first_name","last_name","date_of_birth","insurance_id",
    "admission_date","discharge_date","current_medications"
  ],
  "properties": {
    "patient_id": {
      "type": "string",
      "pattern": "^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[1-5][0-9a-fA-F]{3}-[89abAB][0-9a-fA-F]{3}-[0-9a-fA-F]{12}$"
    },
    "first_name": {"type":["string","null"]},
    "last_name": {"type":["string","null"]},
    "date_of_birth": {"type":["string","null"], "pattern":"^\\d{4}-\\d{2}-\\d{2}$"},
    "insurance_id": {"type":["string","null"]},
    "admission_date": {"type":["string","null"], "pattern":"^\\d{4}-\\d{2}-\\d{2}$"},
    "discharge_date": {"type":["string","null"], "pattern":"^\\d{4}-\\d{2}-\\d{2}$"},
    "current_medications": {"type":"string"}
  },
  "additionalProperties": False
}


In [4]:
import pkg_resources, sys
print("Python:", sys.version)
for p in ["openai","httpx","requests"]:
    try:
        print(p, pkg_resources.get_distribution(p).version)
    except:
        print(p, "NOT INSTALLED")


Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
openai 1.96.1
httpx 0.28.1
requests 2.32.3


In [5]:
import json, re, sqlite3, hashlib, time, uuid
from datetime import datetime
from jsonschema import validate, ValidationError
from pathlib import Path
import tiktoken
from openai import OpenAI

UUID_RE = re.compile(r'^[0-9a-fA-F]{8}-[0-9a-fA-F]{4}-[1-5][0-9a-fA-F]{3}-[89abAB][0-9a-fA-F]{3}-[0-9a-fA-F]{12}$')
DATE_RE = re.compile(r'^\d{4}-\d{2}-\d{2}$')

client = OpenAI()

def build_system_prompt(doc_text: str) -> str:
    return PROMPT_TEMPLATE.replace("{DOCUMENT_TEXT}", doc_text.strip())

def call_llm(system_prompt: str, model: str = MODEL, max_retries=3, sleep=2):
    for attempt in range(1, max_retries+1):
        try:
            resp = client.chat.completions.create(
                model=model,
                temperature=0,
                messages=[
                    {"role":"system","content": system_prompt},
                    {"role":"user","content":"Return ONLY the JSON object."}
                ]
            )
            content = resp.choices[0].message.content.strip()
            # Remove fences if present
            if content.startswith("```"):
                content = content.strip("`")
                if content.startswith("json"):
                    content = content.split("\n",1)[1]
            return content
        except Exception as e:
            if attempt == max_retries:
                raise
            print(f"[Retry {attempt}] Error: {e} -> waiting {sleep}s")
            time.sleep(sleep)

def internal_basic_checks(obj: dict):
    required = ["patient_id","first_name","last_name","date_of_birth","insurance_id",
                "admission_date","discharge_date","current_medications"]
    if set(obj.keys()) != set(required):
        raise ValueError("Unexpected keys present or missing.")
    if not UUID_RE.match(obj["patient_id"]):
        raise ValueError("Invalid UUID format.")
    for k in ["date_of_birth","admission_date","discharge_date"]:
        if obj[k] is not None and not DATE_RE.match(obj[k]):
            raise ValueError(f"Bad date in {k}")
    if not isinstance(obj["current_medications"], str):
        raise ValueError("current_medications must be string.")

def extract_patient(raw_text: str, model: str = MODEL):
    sp = build_system_prompt(raw_text)
    enc = tiktoken.encoding_for_model(model)
    prompt_tokens = len(enc.encode(sp))
    raw = call_llm(sp, model=model)
    try:
        data = json.loads(raw)
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON parse error: {e}\nRaw snippet:\n{raw[:400]}")
    validate(instance=data, schema=SCHEMA)
    internal_basic_checks(data)
    return data, {"raw": raw, "prompt_tokens": prompt_tokens,
                  "prompt_sha256": hashlib.sha256(sp.encode()).hexdigest()[:16]}

def init_db(path="patient_records.db"):
    conn = sqlite3.connect(path)
    conn.execute("""
        CREATE TABLE IF NOT EXISTS patient (
          patient_id TEXT PRIMARY KEY,
          first_name TEXT,
          last_name TEXT,
          date_of_birth TEXT,
          insurance_id TEXT,
          admission_date TEXT,
          discharge_date TEXT,
          current_medications TEXT,
          inserted_at TEXT,
          prompt_hash TEXT,
          model TEXT
        )
    """)
    return conn

def upsert_record(conn, rec, meta, model=MODEL):
    conn.execute("""
        INSERT INTO patient (patient_id, first_name, last_name, date_of_birth, insurance_id,
          admission_date, discharge_date, current_medications, inserted_at, prompt_hash, model)
        VALUES (:patient_id,:first_name,:last_name,:date_of_birth,:insurance_id,
                :admission_date,:discharge_date,:current_medications,:inserted_at,:prompt_hash,:model)
        ON CONFLICT(patient_id) DO UPDATE SET
          first_name=excluded.first_name,
          last_name=excluded.last_name,
          date_of_birth=excluded.date_of_birth,
          insurance_id=excluded.insurance_id,
          admission_date=excluded.admission_date,
          discharge_date=excluded.discharge_date,
          current_medications=excluded.current_medications,
          inserted_at=excluded.inserted_at,
          prompt_hash=excluded.prompt_hash,
          model=excluded.model
    """, {
        **rec,
        "inserted_at": datetime.utcnow().isoformat(timespec="seconds")+"Z",
        "prompt_hash": meta["prompt_sha256"],
        "model": model
    })
    conn.commit()


In [7]:
# Option A: From uploaded file path
sample_path = "inpatient_initial_adult.txt"  # adjust if different
with open(sample_path, "r", encoding="utf-8", errors="ignore") as f:
    raw_doc = f.read()

# Option B: Direct paste
# raw_doc = """<PASTE RAW TEXT HERE>"""

print(raw_doc[:800])



ement&repositoryId=Medical Management&repositoryType=p8&docid=Medical Management (CCE9C7BF-C4AF-4669-BCEO-3C69888A0912}
2 / 16
100% + æ—¥
NYC Health+Hospital
3/12/2025 5:52:27 PM EDT PAGE 1/018 Fax Server
FAX
To: UM DEPT.
Company: FIDELIS CARE NY MCD
Fax: 833-561-0094
Phone:
From: Bissainthe, Bruno
Fax:
Phone:
E-mail:
NOTES:
ht&repositoryld=Medical Management&repositoryType=p8&docid=Medical Management. (CCE9C7BF-C4AF-4669-BCEO-3C69888A0912) (1085
Date:
3/12/25
To:
3 / 16
100%
B
NYC
HEALTH+
HOSPITALS
Fax Transmission
UM DEPT
Phone
Fax: 833-561-0094
From:
Bruno Bissainthe
Case Management / Utilization Review Department
Bellevue Hospital Center
462 1st Ave
New York NY 10016
Phone: 212-562-4404
Fax: 212-562-5008
Comments: This correspondence contains clinical information. Please use the conta


In [8]:
conn = init_db()
record, meta = extract_patient(raw_doc, model=MODEL)
upsert_record(conn, record, meta, model=MODEL)

print("Extracted JSON:")
print(json.dumps(record, indent=2))
print("\nMetadata:", meta)


Extracted JSON:
{
  "patient_id": "f3c1c1b5-1c3e-4c1e-8c3e-1c3e1c3e1c3e",
  "first_name": "Sandro",
  "last_name": "Colon",
  "date_of_birth": "1972-04-27",
  "insurance_id": "74143012700",
  "admission_date": "2025-03-11",
  "discharge_date": null,
  "current_medications": "Zyprexa Zydis 10 mg BID; Depakote 1000 mg BID"
}

Metadata: {'raw': '{\n  "patient_id": "f3c1c1b5-1c3e-4c1e-8c3e-1c3e1c3e1c3e",\n  "first_name": "Sandro",\n  "last_name": "Colon",\n  "date_of_birth": "1972-04-27",\n  "insurance_id": "74143012700",\n  "admission_date": "2025-03-11",\n  "discharge_date": null,\n  "current_medications": "Zyprexa Zydis 10 mg BID; Depakote 1000 mg BID"\n}', 'prompt_tokens': 9000, 'prompt_sha256': '648ea8e30bf2edeb'}


In [9]:
import pandas as pd
df = pd.read_sql_query("SELECT * FROM patient", conn)
df


,patient_id,first_name,last_name,date_of_birth,insurance_id,admission_date,discharge_date,current_medications,inserted_at,prompt_hash,model
0,f3c1c1b5-1c3e-4c1e-8c3e-1c3e1c3e1c3e,Sandro,Colon,1972-04-27,74143012700,2025-03-11,None,Zyprexa Zydis 10 mg BID; Depakote 1000 mg BID,2025-07-19T17:50:27Z,648ea8e30bf2edeb,gpt-4o-mini


In [10]:
from glob import glob

results = []
for path in glob("docs/*.txt"):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read()
    try:
        rec, meta = extract_patient(raw, model=MODEL)
        upsert_record(conn, rec, meta, model=MODEL)
        results.append({"file": path, "patient_id": rec["patient_id"], "ok": True})
    except Exception as e:
        results.append({"file": path, "error": str(e), "ok": False})

import pandas as pd
pd.DataFrame(results)


""


In [11]:
def call_llm_json_mode(system_prompt: str, model: str = MODEL):
    resp = client.chat.completions.create(
        model=model,
        temperature=0,
        response_format={"type":"json_object"},
        messages=[{"role":"system","content": system_prompt}]
    )
    return resp.choices[0].message.content

# Example usage:
# raw = call_llm_json_mode(build_system_prompt(raw_doc))
# data = json.loads(raw)  # proceed with same validation


In [12]:
import math

def estimate_cost(tokens_prompt, tokens_completion, price_in_per_1k=0.01):
    return (tokens_prompt + tokens_completion) / 1000 * price_in_per_1k

# After a call, meta["prompt_tokens"] already stored; you can add completion tokens if you capture them.


In [13]:
from google.colab import files
conn.close()
files.download("patient_records.db")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>